# Implementation of GoogLeNet (Inception)

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, optimizers, Input, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten, LayerNormalization, concatenate
from tensorflow.keras.utils import plot_model
import pydot

(x, y), (x_test, y_test) = datasets.cifar10.load_data()
y = tf.squeeze(y, axis = -1)
y_test = tf.squeeze(y_test, axis = -1)
num_class = 1000

x = 2 * x.reshape(x.shape[0],32,32,3).astype('float32')/255 -1 
x_test = 2 * x_test.reshape(x_test.shape[0], 32, 32, 3).astype('float32')/255 - 1

train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.shuffle(1000).batch(128)
test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_db = test_db.batch(128)

def getInceptionModule(x, filters_11, filters_113, filters_33, filters_115, filters_55, filters_11mp):
    x_11 = Conv2D(filters = filters_11, kernel_size = [1, 1], strides = 1, activation='relu')(x)
    
    x_113 = Conv2D(filters = filters_113, kernel_size = [1, 1], strides = 1, activation='relu')(x)
    x_33 = Conv2D(filters = filters_33, kernel_size = [3, 3], strides = 1, padding = 'same', activation = 'relu')(x_113)
    
    x_115 = Conv2D(filters = filters_115, kernel_size = [1, 1], strides = 1, activation='relu')(x)
    x_55 = Conv2D(filters = filters_55, kernel_size = [5, 5], strides = 1, padding='same', activation='relu')(x_115)
    
    x_mp = MaxPool2D(pool_size = [3, 3], padding = 'same', strides = 1)(x)
    x_mp11 = Conv2D(filters = filters_11mp, kernel_size = [1,1], strides = 1, activation='relu')(x_mp)
    
    return concatenate([x_11, x_33, x_55, x_mp11]) 

googlenet_inputs = Input(shape=[224, 224, 3])
x = Conv2D(64, kernel_size = [7,7], strides = 2, padding = 'same', activation='relu')(googlenet_inputs)
x = MaxPool2D(pool_size=[3,3], strides = 2, padding = 'same')(x)
x = LayerNormalization(epsilon = 1e-6)(x)

x = Conv2D(64, kernel_size=[1,1], activation = 'relu')(x)
x = Conv2D(192, kernel_size=[3,3], padding='same', activation = 'relu')(x)
x = MaxPool2D(pool_size=[3,3], strides = 2)(x)
x = LayerNormalization(epsilon = 1e-6)(x)

inc_3a = getInceptionModule(x, 64, 96, 128, 16, 32, 32)
inc_3b = getInceptionModule(inc_3a, 128, 128, 192, 32, 96, 64)
x = MaxPool2D(pool_size = [3, 3], strides = 2, padding = 'same')(inc_3b)

inc_4a = getInceptionModule(x, 192, 96, 208, 16, 48, 64)
inc_4b = getInceptionModule(inc_4a, 160, 112, 224, 24, 64, 64)
inc_4c = getInceptionModule(inc_4b, 128, 128, 256, 24, 64, 64)
inc_4d = getInceptionModule(inc_4c, 112, 144, 288, 32, 64, 64)
inc_4e = getInceptionModule(inc_4d, 256, 160, 320, 32, 128, 128)
x = MaxPool2D(pool_size = [3, 3], strides = 2, padding = 'same')(inc_4e)

inc_5a = getInceptionModule(x, 256, 160, 320, 32, 128, 128)
inc_5b = getInceptionModule(inc_5a, 384, 192, 384, 48, 128, 128)

avg_pool_6 = AveragePooling2D(pool_size = [7, 7], padding = 'same')(inc_5b)
x = Dropout(rate=0.4)(avg_pool_6)
x = Flatten()(x)
x = Dense(1000, activation='relu')(x)
outputs_7 = Dense(num_class, activation='softmax')(x)

outputs_4a = AveragePooling2D(pool_size= [5, 5], strides = [3,3], padding = 'valid')(inc_4a)
outputs_4a = Conv2D(filters = 128, kernel_size = [1,1], strides = [1,1], padding = 'same', activation='relu')(outputs_4a)
outputs_4a = Dense(1024, activation='relu')(outputs_4a)
outputs_4a = Dropout(0.7)(outputs_4a)
outputs_4a = Dense(num_class, activation='softmax')(outputs_4a)

outputs_4d = AveragePooling2D(pool_size= [5, 5], strides = [3,3], padding = 'valid')(inc_4d)
outputs_4d = Conv2D(filters = 128, kernel_size = [1,1], strides = [1,1], padding = 'same', activation='relu')(outputs_4d)
outputs_4d = Dense(1024, activation='relu')(outputs_4d)
outputs_4d = Dropout(0.7)(outputs_4d)
outputs_4d = Dense(num_class, activation='softmax')(outputs_4d)


googlenet_model = Model(inputs = googlenet_inputs, outputs = [outputs_7, outputs_4a, outputs_4d])

googlenet_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
googlenet_model.summary()

plot_model(googlenet_model, to_file='googlenet_model.png', show_shapes=True, show_layer_names=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 56, 56, 64)   128         max_pooling2d[0][0]              
______________________________________________________________________________________________